# Customize output format in prompt

## User prompt configuration

### [X] Create prompt template

The prompt template is defined in the file:

[/workspace/data/prompts/trading/explain_top_returns/from_date/template.md](/workspace/data/prompts/trading/explain_top_returns/from_date/template.md)

And it will be loaded as follows:

In [1]:
name_prompt = 'trading' #TODO: define your folder
name_template = 'explain_top_returns' #TODO: define your folder

In [2]:
from pathlib import Path

folder_template = f'{name_prompt}/{name_template}' 
folder = Path(f'/workspace/data/prompts/{folder_template}')

path = folder / 'template.md'
with open(path, 'r') as file:
    template = file.read()

template

'On {DATE}, the financial asset with ticker {TICKER} recorded a return of {RETURN}%, calculated as the percentage change between the closing price on that date and the previous corresponding date according to the defined calculation frequency: {FREQUENCY} (for example: daily D, monthly M, quarterly Q, or yearly Y).\n\nExplain the reasons that could justify this extreme movement, using relevant news published around that date. Provide a well-founded explanation, the publication date of the news, and a link to the source.'

### [X] Define custom output format and import it

[/workspace/data/prompts/weather/output_format.py](/workspace/data/prompts/weather/output_format.py)

In [3]:
output_class_name = 'News' #TODO: define your class

In [4]:
from importlib import import_module
OutputParser = getattr(import_module(f'data.prompts.{name_prompt}.output_parser'), output_class_name)

In [5]:
OutputParser

data.prompts.trading.output_parser.News

## Combine template and output format

In [6]:
from modules.prompt import CustomPrompt

custom_prompt = CustomPrompt(template, OutputParser)
prompt = custom_prompt.get_prompt()
prompt

PromptTemplate(input_variables=['DATE', 'FREQUENCY', 'RETURN', 'TICKER'], input_types={}, partial_variables={'FORMAT_INSTRUCTIONS': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"description": "This is the output format for the news prompt.", "properties": {"stock": {"description": "stock symbol", "title": "Stock", "type": "string"}, "date": {"description": "date of the news", "format": "date", "title": "Date", "type": "string"}, "value": {"description": "significant accumulated value of the stock", "title": "Value", "type": "number"}, "title": {"description": "title of the news"

## Chain

### Define model

In [7]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-search-preview")

model

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x734ed22e5490>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x734ed20d16a0>, root_client=<openai.OpenAI object at 0x734ed27b5130>, root_async_client=<openai.AsyncOpenAI object at 0x734ed2333410>, model_name='gpt-4o-search-preview', model_kwargs={}, openai_api_key=SecretStr('**********'))

### Compose chain

In [8]:
chain = prompt | model | custom_prompt.parser
chain

PromptTemplate(input_variables=['DATE', 'FREQUENCY', 'RETURN', 'TICKER'], input_types={}, partial_variables={'FORMAT_INSTRUCTIONS': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"description": "This is the output format for the news prompt.", "properties": {"stock": {"description": "stock symbol", "title": "Stock", "type": "string"}, "date": {"description": "date of the news", "format": "date", "title": "Date", "type": "string"}, "value": {"description": "significant accumulated value of the stock", "title": "Value", "type": "number"}, "title": {"description": "title of the news"

### [X] Preprocess input

In [9]:
TICKER = 'AAPL'
FREQ = 'D'
N_SAMPLES = 5

In [10]:
from modules import utils

In [11]:
from datetime import date
TODAY = date.today().strftime('%Y-%m-%d')

TODAY

'2025-04-20'

In [12]:
r = utils.calculate_significant_returns_simple(
    ticker=TICKER,
    start='2020-01-01',
    end=TODAY,
    freq=FREQ,
    n=N_SAMPLES
)

r

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Date
2025-04-09    15.328853
2020-03-16   -12.864690
2020-03-13    11.980818
2020-07-31    10.468866
2020-03-24    10.032518
Name: AAPL, dtype: float64

### [X] Invoke chain

To get the response, we need to invoke the chain with the input data.

In [13]:
outputs = []

for date, change in r.items():
    try:
        output = chain.invoke({
            'DATE': date.strftime('%Y-%m-%d'),
            'RETURN': round(change, 2),
            'TICKER': TICKER,
            'FREQUENCY': FREQ
        })
        outputs.append(output.model_dump())
    except Exception as e:
        print(f"Error: {e}")
        continue

## Output

### JSON

In [14]:
outputs

[{'stock': 'AAPL',
  'date': datetime.date(2025, 4, 9),
  'value': 15.33,
  'title': "Apple's Stock Surges Amid Tariff-Induced Consumer Rush",
  'url': 'https://www.axios.com/2025/04/09/apple-stock-tariffs-china-iphones',
  'source': 'Axios',
  'explanation': "On April 9, 2025, Apple Inc. experienced a significant stock price increase of 15.33%. This surge was primarily driven by heightened consumer demand for iPhones in the United States. In response to newly imposed tariffs of up to 104% on Chinese imports by the U.S. government, Apple expedited shipments of iPhones to the U.S. to mitigate potential price hikes. Analysts projected that the price of an iPhone 16 Pro Max 256 GB could increase by $350 (29%) to $1,549 due to these tariffs. Consequently, consumers rushed to purchase iPhones before the anticipated price increases took effect, leading to a substantial boost in sales and positively impacting Apple's stock performance."},
 {'stock': 'AAPL',
  'date': datetime.date(2020, 3, 16

### DataFrame

In [15]:
import pandas as pd

df = pd.DataFrame(outputs)

col_num = df.select_dtypes(include=['number']).columns
df.style.format('{:.2f}', subset=col_num)

,stock,date,value,title,url,source,explanation
0,AAPL,2025-04-09,15.33,Apple's Stock Surges Amid Tariff-Induced Consumer Rush,https://www.axios.com/2025/04/09/apple-stock-tariffs-china-iphones,Axios,"On April 9, 2025, Apple Inc. experienced a significant stock price increase of 15.33%. This surge was primarily driven by heightened consumer demand for iPhones in the United States. In response to newly imposed tariffs of up to 104% on Chinese imports by the U.S. government, Apple expedited shipments of iPhones to the U.S. to mitigate potential price hikes. Analysts projected that the price of an iPhone 16 Pro Max 256 GB could increase by $350 (29%) to $1,549 due to these tariffs. Consequently, consumers rushed to purchase iPhones before the anticipated price increases took effect, leading to a substantial boost in sales and positively impacting Apple's stock performance."
1,AAPL,2020-03-16,-12.86,Apple's Stock Plunges Over 12% Amid Retail Store Closures Due to COVID-19,https://9to5mac.com/2020/03/16/aapl-shares-fall-12-at-stock-market-open-as-apple-closes-retail-stores-due-to-coronavirus-concerns/,9to5Mac,"On March 16, 2020, Apple Inc. announced the closure of all its retail stores outside of Greater China until March 27 in response to the escalating COVID-19 pandemic. This decision was part of broader efforts to mitigate the spread of the virus. The closure of retail stores was expected to significantly impact Apple's revenue, as customers would be unable to experience and purchase products in person. Additionally, the pandemic led to reduced consumer demand and supply chain disruptions, further affecting Apple's operations. These factors contributed to a sharp decline in investor confidence, resulting in a 12.86% drop in Apple's stock price on that day."
2,AAPL,2020-03-13,11.98,Why Apple Stock Jumped Today,https://www.nasdaq.com/articles/why-apple-stock-jumped-today-2020-03-13,Nasdaq,"On March 13, 2020, Apple's stock experienced a significant increase of 11.98%. This surge can be attributed to several key developments: Firstly, Apple reopened all 42 of its stores in China, signaling a recovery in one of its major markets. Secondly, the company announced that its annual Worldwide Developers Conference (WWDC) would be held as an online event in June, demonstrating adaptability amid the COVID-19 pandemic. Additionally, the broader market was rebounding from previous sell-offs, contributing to the positive momentum in Apple's stock price."
3,AAPL,2020-07-31,10.47,Apple Reports Record June Quarter Results and Announces 4-for-1 Stock Split,https://www.apple.com/newsroom/2020/07/apple-reports-third-quarter-results/,Apple Newsroom,"On July 30, 2020, Apple announced its fiscal third-quarter results, reporting a revenue of $59.7 billion, an 11% increase from the previous year, and earnings per share of $2.58, up 18%. Additionally, Apple declared a 4-for-1 stock split to make shares more accessible to a broader range of investors. These strong financial results and the stock split announcement likely contributed to the significant 10.47% increase in Apple's stock price on July 31, 2020."
4,AAPL,2020-03-24,10.03,"Markets rally for a second day, pushing Apple’s market cap back above $1 trillion",https://fortune.com/2020/03/25/aapl-apple-stock-market-cap-dow-jones-sp-500-today-news-rally/,Fortune,"On March 24, 2020, Apple's stock price surged by 10.03%, coinciding with a broader market rally. This uptick was largely driven by investor optimism surrounding the U.S. government's proposed $2 trillion stimulus package aimed at mitigating the economic impact of the COVID-19 pandemic. The stimulus plan included direct payments to individuals, enhanced unemployment benefits, and financial support for businesses, which collectively bolstered market confidence. As a result, major indices like the Dow Jones and S&P 500 experienced significant gains, and Apple's market capitalization rebounded above the $1 trillion mark."


### Export to Excel and CSV

In [16]:
from datetime import datetime
import os

# Create a folder with the current datetime
current_datetime = datetime.now().strftime('%Y%m%d_%H%M%S')
output_folder = folder / f'outputs/{current_datetime}'
output_folder.mkdir(parents=True, exist_ok=True)

# Save the files in the newly created folder
df.to_excel(output_folder / 'output.xlsx', index=False)
df.to_csv(output_folder / 'output.csv', index=False)